In [1]:
%pip install pyarrow
%pip install fastparquet
%pip install awswrangler
%pip install s3fs

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.0-py3-none-any.whl (176 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import s3fs
import pandas as pd
import awswrangler as wr


In [3]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")

def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3


In [4]:
# start_date_s = '2024-05-14'
# end_date_s = '2024-05-16'
# date_range = pd.date_range(start=start_date_s, end=end_date_s)
# # Convert the date range to a list of strings
# date_list = date_range.strftime('%Y-%m-%d').tolist()
# date_list

In [25]:
list_files = ["s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-15_2024-05-15_top_10000.parquet/",\
    "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-16_2024-05-16_top_1_9000.parquet/",\
    "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-18_2024-05-18_top_1_91000.parquet/",\
]


In [26]:
path_file = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-15_2024-05-15_top_10000_9000.parquet/"
df_trans = pd.read_parquet(path_file, engine='pyarrow')

In [27]:
df_trans.shape[0]

2465438

In [28]:
df_trans['true_merchant_name_combined'].nunique()

9001

In [32]:
# max_rows_per_type = 10000
# df_total = pd.DataFrame()
# for one_date in date_list:
#     one_date_s = str(one_date)
#     path_file =  f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/trans_{one_date_s}_{one_date_s}"
#     print(f"Load {path_file}")
#     df_trans = pd.read_parquet(path_file, engine='pyarrow')
#     print(f" Loaded shape {df_trans.shape}")
#     sampled_df = df_trans.groupby('merchant_name_combined').apply(lambda x: x.sample(min(len(x), max_rows_per_type))).reset_index(drop=True)
#     print(f" sampled shape {sampled_df.shape}")
#     df_total = pd.concat([df_total,sampled_df ], axis=0)

df_total = pd.DataFrame()
for path_file in list_files:
    # one_date_s = str(one_date)
    # path_file =  f"s3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/trans_{one_date_s}_{one_date_s}"
    print(f"Load {path_file}")
    df_trans = pd.read_parquet(path_file, engine='pyarrow')
    print(df_trans.shape)
    # print(f" Loaded shape {df_trans.shape}")
    # sampled_df = df_trans.groupby('merchant_name_combined').apply(lambda x: x.sample(min(len(x), max_rows_per_type))).reset_index(drop=True)
    # print(f" sampled shape {sampled_df.shape}")
    df_total = pd.concat([df_total,df_trans ], axis=0)
    del df_trans


Load s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-15_2024-05-15_top_10000.parquet/
(2478930, 11)
Load s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-16_2024-05-16_top_1_9000.parquet/
(2634326, 11)
Load s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-18_2024-05-18_top_1_91000.parquet/
(935823, 11)


In [33]:
df_total.shape

(6049079, 11)

In [34]:
df_total.head(10)

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,is_duplicate,original_merchant_name_combined,amount,merchant_name_combined,true_label,label
0,9808946907,Earnin REPAYMENT Yolanda Triplett. Channel: No...,Earnin REPAYMENT Yolanda Triplett. Type: finan...,Earnin REPAYMENT Yolanda Triplett,Earnin,0,Earnin,-105.0,Clr,0,0.072491
1,9815434413,Earnin DES:REPAYMENT ID: INDN:Salise Marin CO ...,Earnin DES:REPAYMENT ID: INDN:Salise Marin CO ...,Earnin DES:REPAYMENT ID: INDN:Salise Marin CO ...,Earnin,0,Earnin,-54.0,Family Dollar,0,0.130614
2,9816255697,Earnin REPAYMENT WEB ID: . Channel: None. Amou...,Earnin REPAYMENT WEB ID: . Type: financial_ins...,Earnin REPAYMENT WEB ID:,Earnin,0,Earnin,-100.0,Dions Quick Mart,0,0.062103
3,9802511831,Earnin Djaah. Channel: None. Amount: -104.5,Earnin Djaah. Type: financial_institution. Cha...,Earnin Djaah,Earnin,0,Earnin,-104.5,Cash App Keevin,0,0.105194
4,9810372468,MONEY TRANSFER AUTHORIZED ON 05/14 FROM Earnin...,MONEY TRANSFER AUTHORIZED ON 05/14 FROM Earnin...,MONEY TRANSFER AUTHORIZED ON 05/14 FROM Earnin...,Earnin,0,Earnin,50.0,Resident Insure,0,0.155062
5,9811887561,RECEIVED RTP ID P1BCLED FROM EARNIN RFIFI_B. C...,RECEIVED RTP ID P1BCLED FROM EARNIN RFIFI_B. T...,RECEIVED RTP ID P1BCLED FROM EARNIN RFIFI_B,Earnin,0,Earnin,10.0,Walgreens,0,0.046423
6,9807255658,Earnin REPAYMENT . Channel: None. Amount: -55.0,Earnin REPAYMENT . Type: financial_institution...,Earnin REPAYMENT,Earnin,0,Earnin,-55.0,A&W,0,0.084778
7,9808648124,RTP REC EARNIN RGFIJ_B US . Channel: None. Amo...,RTP REC EARNIN RGFIJ_B US . Type: financial_in...,RTP REC EARNIN RGFIJ_B US,Earnin,0,Earnin,100.0,Brigit,0,0.050908
8,9810758713,Earnin REPAYMENT WEB ID: . Channel: None. Amou...,Earnin REPAYMENT WEB ID: . Type: financial_ins...,Earnin REPAYMENT WEB ID:,Earnin,0,Earnin,-115.0,Netflix,0,0.046781
9,9820474392,Earnin REPAYMENT WEB ID: . Channel: None. Amou...,Earnin REPAYMENT WEB ID: . Type: financial_ins...,Earnin REPAYMENT WEB ID:,Earnin,0,Earnin,-109.0,McDonald''s,0,0.463215


In [35]:
df_total.shape

(6049079, 11)

In [36]:
df_total['true_merchant_name_combined'].nunique()

93458

In [37]:
import re
pattern = r"Cash\sApp\s(?!Transfer)[\w\s]+"
result = re.sub(pattern, "Cash App", 'Cash App Transfer')
result
df_total['original_merchant_name_combined'] = df_total['original_merchant_name_combined'].apply(lambda x: re.sub(pattern, "Cash App", x) )
df_total['true_merchant_name_combined'] = df_total['true_merchant_name_combined'].apply(lambda x: re.sub(pattern, "Cash App", x))
df_total['merchant_name_combined'] = df_total['merchant_name_combined'].apply(lambda x: re.sub(pattern, "Cash App", x) )

In [38]:
df_total['true_merchant_name_combined'].nunique()

89559

In [39]:
df_merchant_counts = df_total[['true_merchant_name_combined']].value_counts().to_frame()
df_merchant_counts.reset_index(drop=False, inplace=True)
df_merchant_counts

,true_merchant_name_combined,count
0,Cash App,122887
1,Savings,90000
2,Walmart,82542
3,Earnin,81197
4,McDonald''s,80796
...,...,...
89554,Hes Energynet,2
89555,Herzing Universi,2
89556,Hertler.annvisa,2
89557,Hertford County Abc Bo,2


In [42]:
df_merchant_counts[df_merchant_counts['count']<100]

,true_merchant_name_combined,count
3184,Cajun Vending,98
3185,Fast Track,98
3186,Noodles & Company,98
3187,Treasury Tax,98
3188,SchoolCafe,98
...,...,...
89554,Hes Energynet,2
89555,Herzing Universi,2
89556,Hertler.annvisa,2
89557,Hertford County Abc Bo,2


In [43]:
# Define the maximum number of rows per type
max_rows_per_type = 50000
top_tx_merchants = df_merchant_counts[df_merchant_counts['count'] > max_rows_per_type]['true_merchant_name_combined'].tolist()

In [44]:


df_top = df_total[df_total['merchant_name_combined'].isin(top_tx_merchants)]
df_not_top = df_total[~df_total['merchant_name_combined'].isin(top_tx_merchants)]
sampled_df = df_top.groupby('merchant_name_combined').apply(lambda x: x.sample(min(len(x), max_rows_per_type))).reset_index(drop=True)
df_all_sampled = pd.concat([sampled_df,df_not_top], axis=0)


In [45]:
df_total.shape

(6049079, 11)

In [46]:
df_all_sampled.shape

(5615665, 11)

In [48]:
s3_path_out = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/cons_2024-05-15_2024-05-18_1.parquet/"

In [49]:
wr.s3.to_parquet(df = df_all_sampled, path =s3_path_out, dataset=True )

{'paths': ['s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/cons_2024-05-15_2024-05-18_1.parquet/ac45bd41a9274be39fe18728158c8beb.snappy.parquet'],
 'partitions_values': {}}